In [12]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive

In [3]:
drive.mount("/content/gdrive",force_remount=True)

Mounted at /content/gdrive


In [ ]:
!ls 'gdrive/MyDrive/Floor recognition /'

'12 March'		   20211120_141821.jpg	     20211120_153424.jpg
 20211119_150050.jpg	  '20211120_141822(0).jpg'   20211120_153425.jpg
 20211119_150053.jpg	   20211120_141822.jpg	     20211120_153426.jpg
 20211119_150058.jpg	   20211120_141823.jpg	     20211120_153427.jpg
 20211119_150104.jpg	   20211120_141824.jpg	     20211120_153428.jpg
 20211119_150105.jpg	  '20211120_141825(0).jpg'   20211120_153429.jpg
 20211119_150107.jpg	   20211120_141825.jpg	     20211120_153431.jpg
 20211119_150109.jpg	   20211120_141826.jpg	     20211120_153432.jpg
 20211119_150111.jpg	  '20211120_141827(0).jpg'   20211120_153444.jpg
 20211119_150112.jpg	   20211120_141827.jpg	     20211120_153448.jpg
 20211119_150114.jpg	  '20211120_141828(0).jpg'   20211120_153900.jpg
 20211119_150117.jpg	   20211120_141828.jpg	     20211120_153901.jpg
 20211119_150129.jpg	   20211120_141829.jpg	     20211120_153905.jpg
 20211119_150132.jpg	  '20211120_141831(0).jpg'   20211120_153907.jpg
 20211119_150138.jpg	   20211120_14183

In [4]:
%%capture

#!unzip 'gdrive/MyDrive/Floor recognition /Trimix floor.zip' -d 'gdrive/MyDrive/Floor recognition /trimix_images'
#!unzip 'gdrive/MyDrive/Floor recognition /Epoxy floor.zip' -d 'gdrive/MyDrive/Floor recognition /epoxy_images'
#!unzip 'gdrive/MyDrive/Floor recognition /Tiles floor.zip' -d 'gdrive/MyDrive/Floor recognition /tiles_images'
#!unzip 'gdrive/MyDrive/Floor recognition /Floor - Concrete.zip' -d 'gdrive/MyDrive/Floor recognition /concrete_images'

In [13]:
# Importing all necessary libraries
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
 
img_width, img_height = 224, 224

In [ ]:
!pip install pyheif
!pip install whatimage

     |████████████████████████████████| 9.8 MB 8.4 MB/s 


In [ ]:
!ls 'gdrive/MyDrive/Floor recognition /'

'12 March'		   20211120_141821.jpg	     20211120_153424.jpg
 20211119_150050.jpg	  '20211120_141822(0).jpg'   20211120_153425.jpg
 20211119_150053.jpg	   20211120_141822.jpg	     20211120_153426.jpg
 20211119_150058.jpg	   20211120_141823.jpg	     20211120_153427.jpg
 20211119_150104.jpg	   20211120_141824.jpg	     20211120_153428.jpg
 20211119_150105.jpg	  '20211120_141825(0).jpg'   20211120_153429.jpg
 20211119_150107.jpg	   20211120_141825.jpg	     20211120_153431.jpg
 20211119_150109.jpg	   20211120_141826.jpg	     20211120_153432.jpg
 20211119_150111.jpg	  '20211120_141827(0).jpg'   20211120_153444.jpg
 20211119_150112.jpg	   20211120_141827.jpg	     20211120_153448.jpg
 20211119_150114.jpg	  '20211120_141828(0).jpg'   20211120_153900.jpg
 20211119_150117.jpg	   20211120_141828.jpg	     20211120_153901.jpg
 20211119_150129.jpg	   20211120_141829.jpg	     20211120_153905.jpg
 20211119_150132.jpg	  '20211120_141831(0).jpg'   20211120_153907.jpg
 20211119_150138.jpg	   20211120_14183

In [16]:
train_data_dir = 'gdrive/MyDrive/Floor recognition /15 March/Floor Train'
validation_data_dir = 'gdrive/MyDrive/Floor recognition /15 March/Floor Test'
nb_train_samples =208
nb_validation_samples = 79
epochs = 10
batch_size = 16



In [17]:

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [18]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('sigmoid'))


In [22]:
from tensorflow.keras.optimizers import SGD
import tensorflow as tf

In [23]:
model.compile(loss='categorical_crossentropy',
			optimizer=SGD(lr=0.01, momentum=0.9),
			metrics=['accuracy'])


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [24]:
train_datagen = ImageDataGenerator(
	rescale=1. / 255,
	shear_range=0.2,
	zoom_range=0.2,
	horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
	train_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
	validation_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='categorical')

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2)

model.fit_generator(
	train_generator,
	steps_per_epoch=nb_train_samples // batch_size,
	epochs=epochs,
	validation_data=validation_generator,
	validation_steps=nb_validation_samples // batch_size,
	callbacks = [callback]
  )


Found 208 images belonging to 3 classes.
Found 79 images belonging to 3 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
13/13 [==============================] - 102s 7s/step - loss: 1.1138 - accuracy: 0.3125 - val_loss: 1.0937 - val_accuracy: 0.4844
Epoch 2/10
13/13 [==============================] - 40s 3s/step - loss: 1.0937 - accuracy: 0.4183 - val_loss: 1.1280 - val_accuracy: 0.2344
Epoch 3/10
13/13 [==============================] - 40s 3s/step - loss: 1.0746 - accuracy: 0.3798 - val_loss: 1.0501 - val_accuracy: 0.3125
Epoch 4/10
13/13 [==============================] - 41s 3s/step - loss: 1.0337 - accuracy: 0.3990 - val_loss: 1.0092 - val_accuracy: 0.6406


In [25]:
model.save('gdrive/MyDrive/Floor recognition /15 March/model_saved_color.h5')